## Dissertation Early Modelling
### Gavin Rolls

Migrated from OvertureTest.ipynb. I've queried all of my Overture and OSM data in DataCollection.ipynb and done the initial cleaning in DataCleaning.ipynb. This document is dedicated running some early iterations of employment models

In [3]:
#Library Imports

#Basics
import pandas as pd
import geopandas as gpd

#Shapely
from shapely import wkt
import shapely.geometry
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import mapping
from shapely.geometry import shape

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

#Spatial Indexing
import h3

In [5]:
#Import Cleaned Dataset

empl_geog = gpd.read_file('data/combined_data/empl_geog.geojson')

#Reconvert geoformats (Had to remove lists/multipolygons when saving as geojson)

# WKT strings back to Shapely geometries
empl_geog['overture_building_poly'] = empl_geog['overture_building_poly'].apply(lambda x: shapely.wkt.loads(x) if isinstance(x, str) else x)
empl_geog['osm_building_poly'] = empl_geog['osm_building_poly'].apply(lambda x: shapely.wkt.loads(x) if isinstance(x, str) else x)
empl_geog['place_points'] = empl_geog['place_points'].apply(lambda x: shapely.wkt.loads(x) if isinstance(x, str) else x)

# Convert string representations of lists back to lists
empl_geog['category_list'] = empl_geog['category_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

empl_geog.head()

,LSOA11CD,LSOA11NM_x,population,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,...,LSOA11NM_y,total,num_overture_buildings,num_osm_buildings,num_places,overture_building_poly,osm_building_poly,place_points,category_list,geometry
0,E01000001,City of London 001A,1465,lsoa2011:E01000001 : City of London 001A,0,0,0,0,10,0,...,City of London 001A,14360,29,86,243,"MULTIPOLYGON (((-0.0968818 51.5148582, -0.0969...","MULTIPOLYGON (((-0.0983297 51.5158158, -0.0983...","MULTIPOINT ((-0.0987982 51.5176724), (-0.09860...","[pub, coffee_shop, immigration_law, shared_off...","POLYGON ((-0.09729 51.52158, -0.09652 51.52027..."
1,E01000002,City of London 001B,1436,lsoa2011:E01000002 : City of London 001B,0,0,0,0,0,0,...,City of London 001B,45605,40,96,419,"MULTIPOLYGON (((-0.0932921 51.5152927, -0.0932...","MULTIPOLYGON (((-0.0945314 51.5147658, -0.0944...","MULTIPOINT ((-0.0958288 51.5168656), (-0.09565...","[business_manufacturing_and_supply, education,...","POLYGON ((-0.08813 51.51941, -0.08929 51.51752..."
2,E01000003,City of London 001C,1346,lsoa2011:E01000003 : City of London 001C,0,0,0,0,0,0,...,City of London 001C,760,10,29,55,"MULTIPOLYGON (((-0.0959197 51.5205155, -0.0961...","MULTIPOLYGON (((-0.0959637 51.5209148, -0.0959...","MULTIPOINT ((-0.0976057 51.5226912), (-0.09757...","[health_and_medical, hospital, parking, indust...","POLYGON ((-0.09679 51.52325, -0.09647 51.52282..."
3,E01000005,City of London 001E,985,lsoa2011:E01000005 : City of London 001E,0,10,0,0,0,0,...,City of London 001E,22735,39,133,534,"MULTIPOLYGON (((-0.07593 51.5108457, -0.076125...","MULTIPOLYGON (((-0.0757822 51.511071, -0.07577...","MULTIPOINT ((-0.0799241 51.5156172), (-0.07992...","[professional_services, dance_club, insurance_...","POLYGON ((-0.07323 51.51000, -0.07553 51.50974..."
4,E01000006,Barking and Dagenham 016A,1703,lsoa2011:E01000006 : Barking and Dagenham 016A,0,0,0,0,0,0,...,Barking and Dagenham 016A,105,0,280,11,None,"MULTIPOLYGON (((0.085895 51.5386732, 0.0859413...","MULTIPOINT ((0.0856198 51.538706), (0.0858424 ...","[landmark_and_historical_building, constructio...","POLYGON ((0.09115 51.53909, 0.09326 51.53787, ..."


## Basic Models

### Basic Model - Feature Counts (Overture) + Population Only

In [6]:
#Before incorporating any more data into my model, I want to see how well COUNTS alone can predict employment at LSOA level

#Identifying variables (feature variables vs target to predict)
features = ['num_overture_buildings', 'num_places', 'population']
target = 'total'

X = empl_geog[features]
y = empl_geog[target]

#Split into train/test - I know I need to make this spatially sensitive but we're just going with this for now
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Init/train model
model = LinearRegression()
model.fit(X_train, y_train)

#Predictions
y_pred = model.predict(X_test)

#Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 4460587.330033497
R^2 Score: 0.31047839937648836


### Basic Model - Feature Counts (OSM) + Population Only

In [7]:
#Before incorporating any more data into my model, I want to see how well COUNTS alone can predict employment at LSOA level

#Identifying variables (feature variables vs target to predict)
features = ['num_osm_buildings', 'num_places', 'population']
target = 'total'

X = empl_geog[features]
y = empl_geog[target]

#Split into train/test - I know I need to make this spatially sensitive but we're just going with this for now
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Init/train model
model = LinearRegression()
model.fit(X_train, y_train)

#Predictions
y_pred = model.predict(X_test)

#Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 4379260.526694899
R^2 Score: 0.3230499697690361


## Derivation of More Detailed Information

### Encoding POI Category Information

In [8]:
# Encode POI categories in data - using One-Hot encoding

#Explode category lists
empl_geog_exploded = empl_geog.explode('category_list')

#Set up encoder
encoder = OneHotEncoder(sparse_output=False)

#Encode
encoded_categories = encoder.fit_transform(empl_geog_exploded[['category_list']])
encoded_df = pd.DataFrame(encoded_categories, columns=encoder.get_feature_names_out(['category_list']))

empl_geog_exploded = pd.concat([empl_geog_exploded.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Reaggregate
empl_geog_exploded.drop(columns=['category_list'], inplace=True)

# Group by LSOA11CD - sum numeric only
numeric_columns = encoded_df.columns.tolist()
empl_geog_encoded = empl_geog_exploded.groupby('LSOA11CD')[numeric_columns].sum().reset_index()

empl_geog_encoded.head()

,LSOA11CD,category_list_3d_printing_service,category_list_abortion_clinic,category_list_abuse_and_addiction_treatment,category_list_accommodation,category_list_accountant,category_list_acoustical_consultant,category_list_active_life,category_list_acupuncture,category_list_addiction_rehabilitation_center,...,category_list_women's_clothing_store,category_list_women's_health_clinic,category_list_wood_and_pulp,category_list_woodworking_supply_store,category_list_writing_service,category_list_yoga_instructor,category_list_yoga_studio,category_list_youth_organizations,category_list_zoo,category_list_None
0,E01000001,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,E01000002,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,E01000003,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E01000005,0.0,0.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,E01000006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Rich Building Geometry Information

In [9]:
# Function to extract slightly more information about the building footprints contained in each LSOA
def extract_multipolygon_features(multipolygon, lsoa_geometry):
    if isinstance(multipolygon, MultiPolygon):
        polygons = list(multipolygon.geoms)
    elif isinstance(multipolygon, Polygon):
        polygons = [multipolygon]
    else:
        return pd.Series({
            'num_polygons': 0,
            'total_area': 0,
            'total_perimeter': 0,
            'area_ratio': 0,
            'avg_area': 0,
            'avg_perimeter': 0
        })

    num_polygons = len(polygons)
    areas = [polygon.area for polygon in polygons]
    perimeters = [polygon.length for polygon in polygons]
    total_area = sum(areas)
    total_perimeter = sum(perimeters)
    avg_area = total_area / num_polygons if num_polygons > 0 else 0
    avg_perimeter = total_perimeter / num_polygons if num_polygons > 0 else 0

    #Built-up area ratio could result in a div by zero error if there are no buildings in an LSOA so this logic has to be included here
    try:
        area_ratio = total_area / lsoa_geometry.area
    except ZeroDivisionError:
        area_ratio = 0

    return pd.Series({
        'num_polygons': num_polygons,
        'total_area': total_area,
        'area_ratio': area_ratio,
        'total_perimeter': total_perimeter,
        'avg_area': avg_area,
        'avg_perimeter': avg_perimeter
    })


In [10]:
#Apply function above

#Both Building Set Geometry Features
overture_geometry_features = empl_geog.apply(
    lambda row: extract_multipolygon_features(row['overture_building_poly'], row['geometry']),
    axis=1
)
osm_geometry_features = empl_geog.apply(
    lambda row: extract_multipolygon_features(row['osm_building_poly'], row['geometry']),
    axis=1
)

empl_geog_overture_features = pd.concat([empl_geog, overture_geometry_features], axis=1)
empl_geog_osm_features = pd.concat([empl_geog, osm_geometry_features], axis=1)


In [11]:
#Check correct application of features
empl_geog_osm_features.head()

,LSOA11CD,LSOA11NM_x,population,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,...,osm_building_poly,place_points,category_list,geometry,area_ratio,avg_area,avg_perimeter,num_polygons,total_area,total_perimeter
0,E01000001,City of London 001A,1465,lsoa2011:E01000001 : City of London 001A,0,0,0,0,10,0,...,"MULTIPOLYGON (((-0.0983297 51.5158158, -0.0983...","MULTIPOINT ((-0.0987982 51.5176724), (-0.09860...","[pub, coffee_shop, immigration_law, shared_off...","POLYGON ((-0.09729 51.52158, -0.09652 51.52027...",0.966502,6.954275e-07,0.005242,24.0,0.000017,0.125819
1,E01000002,City of London 001B,1436,lsoa2011:E01000002 : City of London 001B,0,0,0,0,0,0,...,"MULTIPOLYGON (((-0.0945314 51.5147658, -0.0944...","MULTIPOINT ((-0.0958288 51.5168656), (-0.09565...","[business_manufacturing_and_supply, education,...","POLYGON ((-0.08813 51.51941, -0.08929 51.51752...",0.722431,4.409557e-07,0.003211,48.0,0.000021,0.154119
2,E01000003,City of London 001C,1346,lsoa2011:E01000003 : City of London 001C,0,0,0,0,0,0,...,"MULTIPOLYGON (((-0.0959637 51.5209148, -0.0959...","MULTIPOINT ((-0.0976057 51.5226912), (-0.09757...","[health_and_medical, hospital, parking, indust...","POLYGON ((-0.09679 51.52325, -0.09647 51.52282...",1.126545,7.602040e-07,0.005130,11.0,0.000008,0.056428
3,E01000005,City of London 001E,985,lsoa2011:E01000005 : City of London 001E,0,10,0,0,0,0,...,"MULTIPOLYGON (((-0.0757822 51.511071, -0.07577...","MULTIPOINT ((-0.0799241 51.5156172), (-0.07992...","[professional_services, dance_club, insurance_...","POLYGON ((-0.07323 51.51000, -0.07553 51.50974...",0.540976,2.474802e-07,0.002667,54.0,0.000013,0.143999
4,E01000006,Barking and Dagenham 016A,1703,lsoa2011:E01000006 : Barking and Dagenham 016A,0,0,0,0,0,0,...,"MULTIPOLYGON (((0.085895 51.5386732, 0.0859413...","MULTIPOINT ((0.0856198 51.538706), (0.0858424 ...","[landmark_and_historical_building, constructio...","POLYGON ((0.09115 51.53909, 0.09326 51.53787, ...",0.179683,1.598686e-07,0.002679,21.0,0.000003,0.056256


### Combine Geometric Features and POI Categories

In [15]:
#Combined Dataset

all_data = pd.merge(empl_geog_osm_features, empl_geog_encoded, on='LSOA11CD')

all_data.head()

,LSOA11CD,LSOA11NM_x,population,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,...,category_list_women's_clothing_store,category_list_women's_health_clinic,category_list_wood_and_pulp,category_list_woodworking_supply_store,category_list_writing_service,category_list_yoga_instructor,category_list_yoga_studio,category_list_youth_organizations,category_list_zoo,category_list_None
0,E01000001,City of London 001A,1465,lsoa2011:E01000001 : City of London 001A,0,0,0,0,10,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,E01000002,City of London 001B,1436,lsoa2011:E01000002 : City of London 001B,0,0,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,E01000003,City of London 001C,1346,lsoa2011:E01000003 : City of London 001C,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,E01000005,City of London 001E,985,lsoa2011:E01000005 : City of London 001E,0,10,0,0,0,0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,E01000006,Barking and Dagenham 016A,1703,lsoa2011:E01000006 : Barking and Dagenham 016A,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## More Models

### (OSM) Geometric Features Sensitive Model (Linear Regression)

In [16]:
# Try again with extra polygon information

#Identifying variables (feature variables vs target to predict)
features = ['num_osm_buildings', 'num_places', 'population', 'num_polygons', 'total_area', 'area_ratio', 'total_perimeter', 'avg_area', 'avg_perimeter']
target = 'total'

X = empl_geog_osm_features[features]
y = empl_geog_osm_features[target]

#Split into train/test - I know I need to make this spatially sensitive but we're just going with this for now
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Init/train model
model = LinearRegression()
model.fit(X_train, y_train)

#Predictions
y_pred = model.predict(X_test)

#Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 4692874.792363291
R^2 Score: 0.27457119456693446


### (OSM) Geometric Features Sensitive Model (Machine Learning - Random Forest)

In [17]:
# Try again with extra polygon information

#Identifying variables (feature variables vs target to predict)
features = ['num_osm_buildings', 'num_places', 'population', 'num_polygons', 'total_area', 'area_ratio', 'total_perimeter', 'avg_area', 'avg_perimeter']
target = 'total'

X = empl_geog_osm_features[features]
y = empl_geog_osm_features[target]

#Split into train/test - I know I need to make this spatially sensitive but we're just going with this for now
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Init/train model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

#Predictions
y_pred = model.predict(X_test)

#Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 3679596.206970011
R^2 Score: 0.43120470947956724


### (OSM) Geometric Features Sensitive Model (Machine Learning - Gradient Boost)

In [18]:
# Try again with extra polygon information

#Identifying variables (feature variables vs target to predict)
features = ['num_osm_buildings', 'num_places', 'population', 'num_polygons', 'total_area', 'area_ratio', 'total_perimeter', 'avg_area', 'avg_perimeter']
target = 'total'

X = empl_geog_osm_features[features]
y = empl_geog_osm_features[target]

#Split into train/test - I know I need to make this spatially sensitive but we're just going with this for now
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Init/train model
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

#Predictions
y_pred = model.predict(X_test)

#Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 3387328.8493394866
R^2 Score: 0.47638366044114155


### POI Category Sensitive Model

In [19]:
#Execute the model

features = ['num_osm_buildings', 'num_places', 'population'] + list(encoded_df.columns)
target = 'total'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data[features], all_data[target], test_size=0.2, random_state=42)

# Initialize the model - Random Forest
model = RandomForestRegressor(random_state=42)

# Train the model

model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"\nMean Squared Error: {mse}")

r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")


Mean Squared Error: 2588295.827148397
R-squared: 0.5998989035353968


### OSM Geometric Features and POI Category Sensitive Model - (Machine Learning - Random Forest)

In [20]:
features = ['num_polygons', 'total_area', 'total_perimeter', 'area_ratio', 'avg_area', 'avg_perimeter', 'num_osm_buildings', 'num_places', 'population'] + list(encoded_df.columns)
target = 'total'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data[features], all_data[target], test_size=0.2, random_state=42)

# Initialize the model - Random Forest
model = RandomForestRegressor(random_state=42)

# Train the model

model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"\nMean Squared Error: {mse}")

r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")


Mean Squared Error: 2437912.1626189244
R-squared: 0.6231453456296518


### OSM Geometric Features and POI Category Sensitive Model - (Machine Learning - Gradient Booster)

In [21]:
features = ['num_polygons', 'total_area', 'total_perimeter', 'area_ratio', 'avg_area', 'avg_perimeter', 'num_osm_buildings', 'num_places', 'population'] + list(encoded_df.columns)
target = 'total'

# Split the dataset - 80/20 train test
X_train, X_test, y_train, y_test = train_test_split(all_data[features], all_data[target], test_size=0.2, random_state=42)

# Initialize the model - Random Forest
model = GradientBoostingRegressor()

# Train the model

model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"\nMean Squared Error: {mse}")

r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")


Mean Squared Error: 2128086.661927124
R-squared: 0.6710384501346558
